## Импорт библиотек

In [ ]:
  # Библиотека с API для рекомендательных моделей
!pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Библиотека с API для рекомендательных моделей
!pip install rankfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Для обработки текста и лемматизации
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import scipy.sparse as sparse
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

# Сохранение и загрузка моделей
import pickle

# Графики
import matplotlib.pyplot as plt
import seaborn as sns

# Библиотека для рекомендаций Implicit
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking

# Библиотека для рекомендаций RankFM
from rankfm.rankfm import RankFM
from rankfm import evaluation

# Векторизация текстов
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import cosine_similarity
import sklearn.preprocessing as pp

# Обработка текста и работа со словарями
import nltk
import pymorphy2
import requests

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Немного модернизируем решение, предложенное в Baseline.

- В исходную функцию для подсчета метрик добавим **количество угаданных книг**

- В функцию перевода датафрейма в csr-матрицу добавим `словари`, чтобы не использовать внешние переменные

In [ ]:
###############################################
# Исходные функции для модели с рекомендациями
##############################################

def contest_metric(df_solution, df_grd, print_report=True):

  """
  Возвращает метрики Recall, Precision, F1-score и количество угаданных книг  
  """

  pred = set(df_solution['chb'] + '_' + df_solution['sys_numb'].values)
  true = set(df_grd['chb'] + '_' + df_grd['sys_numb'].values) 
  recall = len(pred.intersection(true)) / len(true)
  precision = len(pred.intersection(true)) / (20 * len(df_grd['chb'].unique()))
  
  f1_score = 2 * (precision * recall) / (precision + recall)
  true_pairs_count = len(pred.intersection(true))

  if print_report == True:
    print(f"Угаданных книг: {true_pairs_count}")
    print(f"Recall: {round(recall, 5)}")
    print(f"Precision: {round(precision, 5)}")
    print(f"F1-score: {round(f1_score, 5)}")  

  return [true_pairs_count, recall, precision, f1_score]


def df_to_sparse(df, user_index_dict, item_index_dict):
  """
  Создает sparse-матрицу из датафрейма

  Данные о пользователях в столбце 'chb', данные о книгах - в столбце 'sys_numb`
  Внешние словари используются для получения ID каждого пользователя и книги
  """
  row = []
  col = []
  data = []

  for line in df.itertuples():
    row.append(user_index_dict[line.chb])
    col.append(item_index_dict[line.sys_numb])
    data.append(1)
  
  return csr_matrix((data, (row, col)))


## Загрузка файлов

In [ ]:
# Доступ к файлам на colab
import glob
import os

# Библиотека для работы с google drive
from google.colab import drive

# Монтируем диск
drive.mount('/content/drive')

# Смотрим файлы в директории
# !ls "/content/drive/My Drive/Colab Notebooks"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Меняем домашнюю директорию
# для доступа к датасетам
os.chdir("/content/drive/My Drive/Colab Notebooks")

In [ ]:
# Cчитывание данных

# читатели
users = pd.read_csv(
    './users.csv', sep=';', index_col=None, 
     dtype={'age': str, 'chb': str, 'chit_type': str, 'gender': str})

# книги
items = pd.read_csv(
    'items.csv', sep=';', index_col=None, 
     dtype={'author': str, 'bbk': str, 'izd': str, 
           'sys_numb': str, 'title': str, 'year_izd': str})

# история книг в библиотеке
transactions = pd.read_csv(
    'train_transactions_extended.csv', 
    sep=';', index_col=None,
    dtype={'chb': str, 'date_1': str, 'is_printed': str, 
            'is_real': str, 'source': str, 'sys_numb': str, 'type': str})

## ЭТАП 1. ALS - Базовая модель для Colloborative Filtering




### (1.1) Вводные

Результаты первых испытаний Baseline:

1. В целом результаты ALS-модели оказались достаточно низкими 
  - подтвердилась наша гипотеза о новых пользователях

2.  На тесте платформы мы получили F1-Score = 0.000690, а на кросс-валидации 0.00290

  - возможно, стоит изменить подход к разбиению выборки и включить всех читателей в train и test

3. Общее время обучения модели ~2-3 минуты, подбор рекомендаций очень долгий, минимум 6 минут.
---
План работы:

- Подготовить функции для экспериментов с моделями и рекомендаций

- Попробовать улучшить качество и скорость базовой модели




### (1.2) Подготовка данных

#### Словари для маппинга ID

In [ ]:
print("Информация о датасетах библиотеки:\n")
print(f"Кол-во пользователей: {len(transactions['chb'].unique())}")
print(f"Кол-во документов в истории пользователей: {len(transactions['sys_numb'].unique())}")
print(f"Общее кол-во документов: {len(items['sys_numb'].unique())}")

Информация о датасетах библиотеки:

Кол-во пользователей: 16753
Кол-во документов в истории пользователей: 194666
Общее кол-во документов: 354355


In [ ]:
# Удаляем дубликаты
df_ML = transactions[['chb', 'sys_numb']].drop_duplicates()

# Словари для получения идексов user и book (для модели)
find_user_index = {user_name: index for index, user_name 
                   in enumerate(df_ML['chb'].unique())}

find_book_index = {item_name: index for index, item_name 
                     in enumerate(items['sys_numb'].unique())}

# Словари для поиска user и book по индексу (для обратного маппинга)
find_user_chb = {index: user_name for index, user_name 
                    in enumerate(df_ML['chb'].unique())}

find_book_numb = {index: book_name for index, book_name 
                    in enumerate(items['sys_numb'].unique())}

model_dicts = {'user_index': find_user_index ,
               'book_index': find_book_index,
               'user_chb': find_user_chb ,
               'book_numb': find_book_numb,
               }

- Теперь все 4 словаря собраны в одном, так мы их не потеряем и сможем быстро передавать в функции

In [ ]:
# Проверяем, как работают словари

print("Пример поиска со словарем\n")
print("Индекс пользователя '300001020830':",
      f"{model_dicts['user_index']['300001020830']}")

print("Идентификатор пользователя с индексом 5:",
      f"{model_dicts['user_chb'][5]}")

Пример поиска со словарем

Индекс пользователя '300001020830': 12581
Идентификатор пользователя с индексом 5: 100000681262


#### Разделение на выборки

- Для равномерного распределения пользователей по датасетам попробуем добавить параметр `stratify`

In [ ]:
# Предварительно удаляем дубликаты
df_ML = transactions[['chb', 'sys_numb']].drop_duplicates()

# Делим данные на тренировочный и тестовый наборы
train_data, test_data = train_test_split(
    df_ML,
    test_size=0.3, 
    stratify=df_ML['chb'],
    random_state=777)

- Проверим количество пользователей в выборках

In [ ]:
print(f"Кол-во уникальных пользователей: {len(df_ML['chb'].unique())}")
print(f"Кол-во уникальных пользвоателей в выборке для обучения: {len(train_data['chb'].unique())}")
print(f"Кол-во уникальных пользвоателей в выборке для тестирования: {len(test_data['chb'].unique())}")

Кол-во уникальных пользователей: 16753
Кол-во уникальных пользвоателей в выборке для обучения: 16753
Кол-во уникальных пользвоателей в выборке для тестирования: 16753


- Отлично! Такой вариант подойдет для первых экспериментов

- Далее можно будет воспользоваться подходом "leave-one-out"

#### Sparse матрица (users, books)

- Попробуем добавить к матрицам параметр alpha

In [ ]:
# Создаем sparse матрицу для обучающей выборки
train_data_sparse = df_to_sparse(train_data, 
                                 model_dicts['user_index'], 
                                 model_dicts['book_index'])
# Добавляем alpha
alpha = 40
train_data_sparse = (train_data_sparse * alpha).astype('double')

# Матрица для полного train датасета
full_train_sparse = df_to_sparse(transactions,
                                 model_dicts['user_index'], 
                                 model_dicts['book_index'])

full_train_sparse_data = (full_train_sparse * alpha).astype('double')

### (1.3) Обучаем модель

In [ ]:
model = AlternatingLeastSquares(factors=200, random_state=1234)

# Для кросс-валидации
model.fit(train_data_sparse)

# Полная обучающая выборка
# model.fit(full_train_sparse_data)

  0%|          | 0/15 [00:00<?, ?it/s]

- Можно сохранить модель, чтобы сэкномить время и ресурсы

In [ ]:
with open('als_model_alpha_40.pickle', 'wb') as fle:
    pickle.dump(model, fle, protocol=pickle.HIGHEST_PROTOCOL)

### (1.4) Формируем рекомендации


#### Базовый подход




Еще раз протестируем подход к генерации рекомендаций, предложенный в Baseline решении конкурса.

In [ ]:
# Получим рекомендации для конкретного пользователя 
userid = 5000
ids, scores = model.recommend(userid, train_data_sparse[userid], N=30, filter_already_liked_items=False)

# Отобразим рекомендации в DataFrame
top20recom_df = pd.DataFrame({"sys_numb": [find_book_numb[id] for id in ids],
                              "score": scores, 
                              "already_liked": np.in1d(ids, train_data_sparse[userid].indices)})

top20recom_df

,sys_numb,score,already_liked
0,RSL01005451231,0.926025,True
1,RSL01008149020,0.840277,True
2,RSL07000377296,0.829643,True
3,RSL01004330579,0.807117,True
4,RSL01004381224,0.774433,True
5,RSL01002726050,0.754422,True
6,RSL01007498961,0.661444,True
7,RSL01003300384,0.661443,True
8,RSL01001653118,0.661443,True
9,RSL01002447381,0.661441,True


In [ ]:
# Поиск рекомендаций для одного пользователя
def get_recom(userid):
  ids, scores = model.recommend(userid, train_data_sparse[userid], 
                                N=20, filter_already_liked_items=True) 

  top20recom_df = pd.DataFrame({
      "sys_numb": [model_dicts['book_numb'][id] for id in ids], 
      "score": scores,
      "already_liked": np.in1d(ids, train_data_sparse[userid].indices)})
  
  return top20recom_df['sys_numb'].values

In [ ]:
# Подбор рекомендаций для всех пользователей 

def get_all_recommendations_baseline (user_item_sparse_data):
  all_rec = []

  for userid in tqdm(range(user_item_sparse_data.shape[0])):    
    user_chb = model_dicts['user_chb'][userid]
    user_rec = get_recom(userid)
    for rec in user_rec:
      all_rec.append([user_chb, rec])

  return all_rec

In [ ]:
# Генерируем рекомендации (базовая реализация)
baseline_recommendations = pd.DataFrame(
    get_all_recommendations_baseline(train_data_sparse),
    columns=["chb", "sys_numb"])

  0%|          | 0/16753 [00:00<?, ?it/s]

**Вывод**

- На генерацию рекомендаций ушло ~8 минут!

- Для полной обучающей выборки времени потребуется еще больше

#### Ускоряем генерацию рекомендаций

1. Первая имплементация функции обучения работает очень медленно

  - будем использовать встроенную функцию  `recommend` со списком пользователей
  - для получения массива рекомендаций [user1_id, [item1_id, item2_id, ..]] 

2. Для оценки качества модели нам нужен список пар [user_id, item_id]. 

  - Переведем 2D-массив в 1D с помощью ravel (без циклов и создания копий)
  - Добавим столбец с идентификаторами пользователей

3. Раскодируем идентификаторы и сравним результат с набором предсказаний из исходной функции.

In [ ]:
# Генерируем набор из ТОП-20 рекомендаций
# для всех пользователей (на входе подаем список индексов)
ids, scores = model.recommend(list(range(0, len(model_dicts['user_index']))), 
                              train_data_sparse, 
                              N=20, filter_already_liked_items=True)

  

- Получили рекоменадации меньше чем за минуту, в 8-10 раз быстрее!

In [ ]:
# Переводим 2D набор из рекомендованных книг в 1D список
# Важен порядок - 20 книг для 1-го пользователя, 20 книг для 2-го пользователя и т.д.
user_item_predictions = pd.DataFrame(ids.ravel(), columns=['book_id'])
user_item_predictions.head(3)

,book_id
0,7592
1,78088
2,383


In [ ]:
# Добавляем индексы пользователей (в порядке, заданном исходной матрицей user-book)
# первые 20 книг - 0, следующие 20 книг - 1 и т.д.
user_item_predictions['user_id'] = user_item_predictions.index  // 20

print("Уникальные индексы пользователей")
print(user_item_predictions.user_id.unique())
print()

user_item_predictions.head(3)

Уникальные индексы пользователей
[    0     1     2 ... 16750 16751 16752]



,book_id,user_id
0,7592,0
1,78088,0
2,383,0


In [ ]:
# Добавляем столбцы с исходными идентификаторами книг и пользователей 
# используем маппинг по индексам
user_item_predictions['sys_numb'] = user_item_predictions['book_id'].map(model_dicts['book_numb'])
user_item_predictions['chb'] = user_item_predictions['user_id'].map(model_dicts['user_chb'])

- Прежде чем использовать новый вариант функции, убедимся, сравним результаты)

In [ ]:
# Объединяем результаты обеих функций
pd.concat([
    baseline_recommendations, 
    user_item_predictions[['chb', 'sys_numb', 'user_id', 'book_id']]
], axis=1)


,chb,sys_numb,chb,sys_numb,user_id,book_id
0,100000641403,RSL01002840579,100000641403,RSL01002840579,0,7592
1,100000641403,RSL01007980442,100000641403,RSL01007980442,0,78088
2,100000641403,RSL01001848685,100000641403,RSL01001848685,0,383
3,100000641403,RSL07000445436,100000641403,RSL07000445436,0,7433
4,100000641403,RSL01002805785,100000641403,RSL01002805785,0,195640
...,...,...,...,...,...,...
335055,400001035059,RSL01010152161,400001035059,RSL01010152161,16752,238294
335056,400001035059,RSL01007980442,400001035059,RSL01007980442,16752,78088
335057,400001035059,RSL01003016669,400001035059,RSL01003016669,16752,15056
335058,400001035059,RSL01000911620,400001035059,RSL01000911620,16752,114828


### (1.5) Оценка качества и сохранение результатов

In [ ]:
# Считаем метрики (f1-score, precision и recall)
_ = contest_metric(baseline_recommendations, test_data)

Угаданных книг: 597
Recall: 0.00869
Precision: 0.00178
F1-score: 0.00296


In [ ]:
# Датафрейм с рекомендациями
solution = user_item_predictions[['chb', 'sys_numb']]
len(solution)

335060

In [ ]:
# Сохраняем предсказания в csv файл
solution.to_csv("recs_train_als_alpha_40.csv", index=False, sep=';')

"""
# Загружаем результаты
user_item_predictions = pd.read_csv(
    "recs_train_als_alpha_40.csv", sep=';',
    dtype={'chb':str, 'sys_numb':str}
)
"""

**Вывод**

- На тесте платформы мы получили более высокий результат f1-score = 0.00558

- Сохранили результат предсказаний для новых экспериментов (с текущими параметрами модели) 

### => Набор новых функций <=


Запишем все шаги, которые мы протестировали, в итоговый набор функций для дальнейших экспериментов.

In [ ]:
##############################################################################
# Функции для работы с библиотекой Implicit и методами Colloborative Filtering
##############################################################################

def matrix_recs_to_pairs(recs_matrix_df, 
                         user_col_name, item_col_name):
  """
  Переводит матрицу рекомендаций в датафрейм с парами user-item
  
  
  Параметры:
  --------
  recs_matrix_df - каждая строка [u]- список рекомендованных 
                   items для пользователя [u]  
  user_col_name - название столбца для user
  item_col_name - название столбца для item

  Возвращает датафрейм с двумя столбцами.
  """
  # Переводим 2D набор items в 1D список
  pairs_df= pd.DataFrame(recs_matrix_df.values.ravel(), 
                         columns=[item_col_name])
  
  # Количество столбцов = количество рекомендаций
  top_n = recs_matrix_df.shape[1]

  # Добавляем индексы пользователей по порядку
  # одинаковый индекс для каждых top_n строк
  pairs_df[user_col_name] = pairs_df.index  // top_n

  
  # Маппинг индексов в имена пользователей
  # используем матрицу рекомендаций 
  pairs_df[user_col_name] = pairs_df[user_col_name].map(
      (recs_matrix_df.reset_index()
       .rename(columns={'index':user_col_name})[user_col_name].to_dict())
  )

  return pairs_df


def recs_stratified_split(df, user_col, test_size, random_state=777):
  """
  Возвращает train и test, разделенные по полю user_col
  """

  # Делим данные на тренировочный и тестовый наборы
  train, test = train_test_split(df, 
                                 test_size=test_size,
                                 stratify=df[user_col],
                                 random_state=random_state)
  return train, test

def get_model_scores(df_solution, df_test):

  """Возвращает список метрик - Recall, Precision и F1-score """

  pred = set(df_solution['chb'] + '_' + df_solution['sys_numb'].values)
  true = set(df_test['chb'] + '_' + df_test['sys_numb'].values)

  recall = len(pred.intersection(true)) / len(true)
  precision = len(pred.intersection(true)) / (20 * len(df_test['chb'].unique()))
  
  f1_score = 2 * (precision * recall) / (precision + recall)
  
  return [round(recall, 5), round(precision,5), round(f1_score, 5)]


def get_item_recommendations(implicit_model, 
                             user_name_dict, item_name_dict,                               
                             user_item_matrix,
                             user_col='user', item_col='item',                            
                             top_n=20):  
  """
  Возвращает ТОП-N рекомендаций, используя обученную модель implicit
  """
  
  # Список идентификаторов всех пользователей
  # в user_item_matrix
  # предполагается, что это тот же набор, на котором обучалась модель
  user_ids_list = list(range(0, user_item_matrix.shape[0]))
 
  # Получаем набор рекомендаций для пользователей в user_ids_list
  # Пары (user-item = 1) из матрицы user_item_matrix исключаются
  ids, scores = implicit_model.recommend(user_ids_list, 
                                         user_item_matrix, N=top_n, 
                                         filter_already_liked_items=True)

  # Переводим 2D набор items в 1D список
  recommendations = pd.DataFrame(ids.ravel(), columns=['item_id'])

  # Добавляем индексы пользователей по порядку
  # одинаковый индекс для каждых top_n строк
  recommendations['user_id'] = recommendations.index  // top_n

  # Выполняем обратное кодирование user_id -> user, book_id -> book
  # Сохраняем в отдельных столбцах
  recommendations[user_col] = recommendations['user_id'].map(user_name_dict)
  recommendations[item_col] = recommendations['item_id'].map(item_name_dict)
  

  return recommendations

##############################################################################
# Функции для моделирования
##############################################################################

def als_model_experiment(data, 
                         mapping_dicts,
                         test_size=0.2, 
                         alpha_value=40, factors_count=200, top_n=10):
  
  """
  Обучает ALS модель на датасете data (с разбиением выборок)  

  Возвращает ТОП-20 рекомендаций и результаты на тесте
  """

  # Разделяем выборки 
  train, test = recs_stratified_split(data, 'chb',test_size)
  
  # Создаем sparse матрицу для обучающей выборки
  train_sparse_matrix = df_to_sparse(train, 
                                     mapping_dicts['user_index'],
                                     mapping_dicts['book_index'])
  
  # Умножаем матрицу на alpha_value
  train_sparse_matrix = (train_sparse_matrix * alpha_value).astype('double')

  # Обучаем модель
  als_model = AlternatingLeastSquares(factors_count, random_state=1234)
  als_model.fit(train_sparse_matrix)
  
  # Генерируем рекомендации с помощью модели
  # Получаем набор пар {user-chb, book-sys_numb}
  predicted_books = get_item_recommendations(als_model, 
                                             mapping_dicts['user_chb'],
                                             mapping_dicts['book_numb'],                                              
                                             train_sparse_matrix,
                                             'chb', 'sys_numb',
                                             top_n=top_n)      
                                             
  # Считаем качество модели
  # и выводим результат
  contest_metric(predicted_books, test)

  return predicted_books, get_model_scores(predicted_books, test), test 


def get_als_recommendations(data, 
                            mapping_dicts, 
                            alpha_value=40, factors_count=200, top_n=20):  
  """
  Обучает ALS модель на датасете data 

  Возвращает ТОП-20 рекомендаций
  """


  # Создаем sparse матрицу для обучения
  train_sparse_matrix = df_to_sparse(data, 
                                     mapping_dicts['user_index'],
                                     mapping_dicts['book_index'])
  
  # Умножаем матрицу на alpha_value
  train_sparse_matrix = (train_sparse_matrix * alpha_value).astype('double')

  # Обучаем модель
  als_model = AlternatingLeastSquares(factors_count, random_state=1234)
  als_model.fit(train_sparse_matrix)
  
  # Генерируем рекомендации с помощью модели
  # Получаем набор пар {user-chb, book-sys_numb}
  predicted_books = get_item_recommendations(als_model, 
                                             mapping_dicts['user_chb'],
                                             mapping_dicts['book_numb'],                                              
                                             train_sparse_matrix,
                                             'chb', 'sys_numb', top_n=top_n)    

  return predicted_books         

- Проверим работу функции для экспериментов с ALS моделью

In [ ]:
# Обучаем ALS-модель с подобранными параметрами 
all_users_predictions, results, _ = als_model_experiment(
    transactions[['chb', 'sys_numb']].drop_duplicates(), 
    model_dicts, top_n=20
)

  0%|          | 0/15 [00:00<?, ?it/s]

Угаданных книг: 467
Recall: 0.0102
Precision: 0.00162
F1-score: 0.00279


- Метрика ниже, но это потому, что мы добавили новую функцию для разделения выборок

- В каждом датасете одинаковое количество пользователей (параметр `stratify`)

In [ ]:
# Сохраняем метрики на кросс-валидации
results_df = pd.DataFrame(index=['Recall', 'Precision', 'F1-score'])
results_df['als_tuned'] = results

**Вывод**

- Получили хороший набор функций для будущего **класса**!



### (1.6) Общий вывод


In [ ]:
results_df

,als_tuned
Recall,0.01020
Precision,0.00162
F1-score,0.00279


1. Мы улучшили базовую модель ALS (библиотека implicit):

  - Оптимизировали генерацию рекомендаций - `скорость` стала в 8 раз выше

  - Удалили **дубликаты** 
  
  - Изменили параметр `filter_already_liked_items=True` (используем для предсказаний только новые книги)

  - Убедились в возможностях тюнинга модели (features_count = 20 и добавление параметра **alpha** = 40)

2. Сформировали набор готовых функций для работы


3. Подтвердилась наша гипотеза, что `Collaborative Filtering` не может дать высоких результатов из-за нехватки данных о чтении книг

  - Можно попробовать выделить **группы активных** пользователей и сделать предсказания только для них, а для остальных использовать другой подход.


## ЭТАП 2. Factorization Machines. Битва с ALS

### Вводные

Для рекомендательной системы на базе Colloborative Filtering мы использовали один из алгоритмов Matrix Factorization - ALS.

  - Результаты были досточно низкие, так как система еще не успела накопить достаточное количество данных о взаимодействии читателей и книг.

- Есть идея добавить в обучение особенности читателей и книг - возраст, авторы книг и т.д.


Такой подход в CF можно реализовать с помощью другой популярной техники -  Factorization Machines.

---

Мы воспользуемся библиотекой RankFM, где они на примере показывают более высокие результаты по сравнению с ALS в библиотеке Implicit 

ШАГ 1. Снова построим модель ALS, но возьмем реализацию, предложенную разработчиками RankFM.
  - Сравним результаты с нашей имплементацией (скорость и качество)
 
ШАГ 2. Построим модель RankFM на базе loss-функции BPR
  - Сравним результаты с ALS.

ШАГ 3. Добавим в модель RankFM признаки  - возраст читателей и год выпуска книг:)
  
  - Надеемся получить более высокий результат или новые инсайты)
---
На всех этапах мы теперь будем параллельно отслеживать метрику hit-rate (мы убедились, что это важный показатель).

Также усредненные precision и recall, чтобы лучше оценить качество "персональных" рекомендаций.

Вперед!


### (2.1) Новая имплементация ALS






In [ ]:
# Оставляем дубликаты
unique_transactions = transactions

# Размер валидационной выборки
valid_pct = 0.25

# Используем np.random для рандомизации индексов
unique_transactions['random'] = np.random.random(size=len(unique_transactions))

# Создаем маски индексов для train и valid датасетов
# в зависимости от valid_pct - размер валидационной выборки
train_mask = unique_transactions['random'] <  (1 - valid_pct)
valid_mask = unique_transactions['random'] >= (1 - valid_pct)

# Разделяем датасет на train и valid, используя маски индексов
transactions_train = unique_transactions[train_mask].groupby(['chb', 'sys_numb']).size().to_frame('books').reset_index()
transactions_valid = unique_transactions[valid_mask].groupby(['chb', 'sys_numb']).size().to_frame('books').reset_index()

# Нормализация, если бы мы учитывали, сколько раз читатели повторно брали книги
sample_weight_train = np.log2(transactions_train['books'] + 1)

print("Количество пользователей с 1, 2, 3-мя любимыми книгами:")
display(transactions_train.books.value_counts())
print()
print("Показатели после нормализации с np.log2")
display(sample_weight_train.value_counts())



Количество пользователей с 1, 2, 3-мя любимыми книгами:


1     162363
2      10676
3       1757
4        553
5        230
6        108
7         57
8         38
9         19
10        13
12         6
15         4
17         3
13         3
11         3
18         2
14         2
24         1
29         1
20         1
23         1
36         1
Name: books, dtype: int64


Показатели после нормализации с np.log2


1.000000    162363
1.584963     10676
2.000000      1757
2.321928       553
2.584963       230
2.807355       108
3.000000        57
3.169925        38
3.321928        19
3.459432        13
3.700440         6
4.000000         4
4.169925         3
3.807355         3
3.584963         3
4.247928         2
3.906891         2
4.643856         1
4.906891         1
4.392317         1
4.584963         1
5.209453         1
Name: books, dtype: int64

**Вывод**

1. У нас есть пул читателей, которые заказывали одну и ту же книгу по несколько раз:

  - возможно, заканчивался срок возврата книги
  - это любимые книги читателя
  - или какой-то сбой при загрузке книг


2. В прошлой имплементации удаление дубликатов снижало качество рекомендаций

  - Можно попробовать провести обучение еще раз, но с нормализацией

  - На первом этапе мы будем действовать как раньше, оставив только уникальные книги для каждого читателя

#### Разделение датасетов 

In [ ]:
unique_transactions = transactions[['chb', 'sys_numb']].drop_duplicates()

# unique_transactions = transactions

# Размер валидационной выборки
valid_pct = 0.2

# Используем np.random для рандомизации индексов
unique_transactions['random'] = np.random.random(size=len(unique_transactions))

# Создаем маски индексов для train и valid датасетов
# в зависимости от valid_pct - размер валидационной выборки
train_mask = unique_transactions['random'] <  (1 - valid_pct)
valid_mask = unique_transactions['random'] >= (1 - valid_pct)

# Разделяем датасет на train и valid, используя маски индексов
transactions_train = unique_transactions[train_mask].groupby(['chb', 'sys_numb']).size().to_frame('books').reset_index()
transactions_valid = unique_transactions[valid_mask].groupby(['chb', 'sys_numb']).size().to_frame('books').reset_index()


# (!!!) для обучения полного датасета и проверки на тесте
transactions_all = unique_transactions.groupby(['chb', 'sys_numb']).size().to_frame('books').reset_index()


# Нормализация, если бы мы учитывали, сколько раз читатели повторно брали книги
# Без дубликатов здесь останется 1 у всех пользователей
sample_weight_train = np.log2(transactions_train['books'] + 1)
sample_weight_valid = np.log2(transactions_valid['books'] + 1)
sample_weight_all = np.log2(transactions_all['books'] + 1)

# Оставляем в отдельных датасетах пары user-book для модели
transactions_train = transactions_train[['chb', 'sys_numb']]
transactions_valid = transactions_valid[['chb', 'sys_numb']]
transactions_all = transactions_all[['chb', 'sys_numb']]

# Формируем списки уникальных пользователей для train и valid
train_users = np.sort(transactions_train['chb'].unique())
valid_users = np.sort(transactions_valid['chb'].unique())

# Новые пользователи библиотеки (незнакомые для модели)
cold_start_users = set(valid_users) - set(train_users)

# Формируем списки уникальных книг для train и valid
train_books = np.sort(transactions_train['sys_numb'].unique())
valid_books = np.sort(transactions_valid['sys_numb'].unique())

# Новые книги, которые взяли читать (незнакомые для модели)
cold_start_books = set(valid_books) - set(train_books)

In [ ]:
print("Неизвестных пользователей для модели:", len(cold_start_users))
print("Неизвестных книг для модели:", len(cold_start_books))

Неизвестных пользователей для модели: 95
Неизвестных книг для модели: 35006


**Вывод**

1. Мы будем испытывать новый подход к разделению выборок (спрячем часть пользователей и книг из выборки)
  - когда в библиотеку добавляются новые пользователи 
  - когда появляются новые книги в базе данных transactions 

2. Такой подход сильнее приближен к реальности и отражает известную проблему рекомендательных систем `cold start`

#### Словари с индексами пользователей и книг 

- В `transactions_train` у нас только пары читатель-книга, а в `sample_weight` - данные о прочитанных книгах.

- Если обучаем модель без дубликатов, то в данных у всех пользователей будет просто 1.

In [ ]:
transactions_train.head(3)

,chb,sys_numb
0,100000641403,RSL01004206702
1,100000641403,RSL01004211574
2,100000644359,RSL01003557352


In [ ]:
sample_weight_train.head(3)

0    1.0
1    1.0
2    1.0
Name: books, dtype: float64

Как и в прошлой реализации, для ALS модели нам понадобятся словари с маппингом идентификаторов:

- В этот раз мы построим словари на базе типа `Series` 

- Перевод в csr-матрицу сделаем без циклов, добавив поля с идентификаторами в обучающий датафрейм


In [ ]:
transactions_train = transactions_all

# Cловари для маппинга индексов ID -> читатель/книга
index_to_user = pd.Series(np.sort(np.unique(transactions_train['chb'])))
index_to_item = pd.Series(np.sort(np.unique(transactions_train['sys_numb'])))

# Словари для обратного маппинга
user_to_index = pd.Series(data=index_to_user.index, index=index_to_user.values)
item_to_index = pd.Series(data=index_to_item.index, index=index_to_item.values)

# Добавляем индексы в обучающий датасет
transactions_train_imp = transactions_train.copy()
transactions_train_imp['user_id'] = transactions_train['chb'].map(user_to_index)
transactions_train_imp['book_id'] = transactions_train['sys_numb'].map(item_to_index)

# Данные для перевода датасета в CSR матрицу
# data = sample_weight_train
data = sample_weight_all
rows = transactions_train_imp['user_id']
cols = transactions_train_imp['book_id']

# CSR матрица user(строки)-books(столбцы)
user_items_imp = csr_matrix((data, (rows, cols)), 
                            shape=(len(user_to_index), len(item_to_index)))

print("Размер sparse-матрицы user-book:", user_items_imp.shape)


Размер sparse-матрицы user-book: (16753, 194666)


#### Обучение модели

1. Будем использовать 200 latent features (скрытые признаки читателей и книг, которые находит модель) 

  - Этот параметр дал нам хороший результат в прошлой реализации ALS

2. Вместо метода recommend здесь используется `recommend_all`:
  - в новой версии разработчики планируют убрать этот метод (объединить все варианты в одной функции `recommend`) 



In [ ]:
alpha_value = 40

# Обучаем модель
imp_model = AlternatingLeastSquares(factors=200, random_state=777)
imp_model.fit((user_items_imp * alpha_value).astype('double'))

  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
# Получаем список рекомендаций - матрица с индексами книг и читателей
recs_imp = imp_model.recommend_all(
    user_items=(user_items_imp * alpha_value).astype('double'), 
    N=20, 
    filter_already_liked_items=False
)

# Создаем датафрейм
# Переводим индексы в исходные идентификаторы
recs_imp = (
    pd.DataFrame(recs_imp, index=index_to_user.values)
    .apply(lambda c: c.map(index_to_item))
)

print("Предсказания модели ALS:")
recs_imp.head(5)

Предсказания модели ALS:


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
100000641403,RSL01004185091,RSL01003441482,RSL01004190969,RSL01008477654,RSL01004211574,RSL01003792529,RSL01003639696,RSL01004191071,RSL01004186571,RSL01003674890,RSL02000025560,RSL01003783075,RSL01004462230,RSL01004186500,RSL01003589346,RSL01007528691,RSL02000000555,RSL01003601634,RSL01004196026,RSL01003730215
100000644359,RSL01009833577,RSL01010556531,RSL01003557352,RSL01010647947,RSL01010278914,RSL01010559974,RSL01010586255,RSL01010587317,RSL01010136617,RSL01009800093,RSL01010594911,RSL01010586376,RSL01010251521,RSL01010456769,RSL07000458969,RSL01010705965,RSL01010553020,RSL01010259357,RSL01010247222,RSL01009839571
100000665127,RSL01002720656,RSL01009973730,RSL01003248275,RSL01003947249,RSL01000198239,RSL01003276143,RSL01003947258,RSL01003947256,RSL01002973126,RSL01003947254,RSL01010117433,RSL01003947250,RSL01003947260,RSL01003915650,RSL01003947253,RSL01003720410,RSL01002881135,RSL01001751573,RSL01004428038,RSL01004914143
100000676191,RSL01003617616,RSL01003622825,RSL01005076342,RSL01003670263,RSL01003910682,RSL01003603366,RSL01003722702,RSL01003634246,RSL01003755071,RSL01003441485,RSL01003682625,RSL01003780913,RSL01003647702,RSL01003553120,RSL01003874690,RSL01003991446,RSL01003890490,RSL01003660926,RSL01002509210,RSL01003773521
100000679200,RSL01010248423,RSL01000123386,RSL01005054479,RSL01003289018,RSL01003177895,RSL01003462568,RSL01008825957,RSL01007884218,RSL01004869991,RSL01004319337,RSL01007503068,RSL01006772159,RSL01008656366,RSL01009774313,RSL01002679312,RSL01003127253,RSL01007843528,RSL07000420608,RSL01006507384,RSL01006059487


**Вывод**

1. Предсказания сформировались очень быстро:

  -  мы не формировали дополнительно наборы пар (user, book), поэтому скорость значительно выше

  - если при расчете метрик производительность сохранится, это будет лучшей реализацией

2. Также мы получили датафрейм, который наглядно визуализирует рекомендации. Особенно, если у нас в голове встроенный кодировщик названий книг)))

Осталось рассчитать метрики, посмотрим, ждет ли нас сюрприз)

#### Оценка результатов

##### Усредненные метрики и hit-rate



In [ ]:
# Словарь со списком книг для каждого читателя
valid_user_items = (
    transactions_valid
    .groupby('chb')['sys_numb']
    .apply(set).to_dict()
)

all_users = set(train_users) & set(valid_users)

# Считаем усредненные метрики
hitrate = np.mean([int(len(set(recs_imp.loc[u]) & valid_user_items[u]) > 0) for u in all_users])

precision = np.mean([
    len(set(recs_imp.loc[u]) & valid_user_items[u]) / 
    len(recs_imp.loc[u]) for u in all_users])

recall = np.mean([
    len(set(recs_imp.loc[u]) & valid_user_items[u]) / 
    len(valid_user_items[u]) for u in all_users])

f1_score = 2 * (precision * recall) / (precision + recall)

print("Усредненные метрики по пользователям:\n")
print(f"Hit-Rate: {round(hitrate, 5)}")
print(f"Recall: {round(recall, 5)}")
print(f"Precision: {round(precision, 5)}")
print(f"F1-score: {round(f1_score, 5)}")

Усредненные метрики по пользователям:

Hit-Rate: 0.741
Recall: 0.63701
Precision: 0.09653
F1-score: 0.16765


**Вывод**

1. Новый подход к реализации однозначно самый производительный

2. Cравним с исходными метриками, где мы оценивали общее количество "угадываний"
  - нам снова нужно будет перевести матрицу рекомендации в пары user-book


##### Сравнение с исходной метрикой f1-score


In [ ]:
# Переводим 2D набор items в 1D список
recs_pairs= pd.DataFrame(recs_imp.values.ravel(), columns=['sys_numb'])

# Добавляем индексы пользователей по порядку
# одинаковый индекс для каждых top_n строк
recs_pairs['chb'] = recs_pairs.index  // 20

recs_pairs.head(3)

,sys_numb,chb
0,RSL01004185091,0
1,RSL01003441482,0
2,RSL01004190969,0


In [ ]:
# Используем обучающий датафрейм для обратного маппинга индексов
recs_pairs['chb'] = recs_pairs['chb'].map(
    recs_imp.reset_index().rename(columns={'index':'chb'}).chb.to_dict()
)
recs_pairs.head(3)

,sys_numb,chb
0,RSL01004185091,100000641403
1,RSL01003441482,100000641403
2,RSL01004190969,100000641403


In [ ]:
print('Общие метрики для "угадывания" пар значений user-book:')
_ = contest_metric(recs_pairs, transactions_valid)

Общие метрики для "угадывания" пар значений user-book:
Угаданных книг: 23113
Recall: 0.50276
Precision: 0.09598
F1-score: 0.16118


**Вывод**

1. Средние показатели precision и f1-score почти совпали с исходными метриками

2. Усредненный recall показывает более оптимистичные результаты

3. Преимущество нового подхода:

  - не нужно переводить датафрейм в пары предсказаний user-book




#### => Набор новых функций <=




In [ ]:
##################################################################
# Подготовка данных для обучения
##################################################################

def recs_split_random(df, test_size):
  """
  Возвращает train и test, разделенные c помощью np.random
  """

  # Используем np.random для рандомизации индексов
  df['random'] = np.random.random(size=len(df))

  # Создаем маски индексов для train и test датасетов
  # test_size - размер валидационной выборки
  train_mask = df['random'] <  (1 - test_size)
  test_mask = df['random'] >= (1 - test_size)
  
  # Разделяем датасет на train и test, используя маски индексов
  train = df[train_mask]
  test = df[test_mask]
  return train, test


def split_and_prepare_recs_data(data, user_col, item_col, test_size,
                                weight_col='items', 
                                print_report=True,
                                delete_test_duplicates=True):
  
  """
  Разделяет на выборки, готовит данные для рекомендательных моделей 
  и выводит количество неизвестных user и item в тесте

  Возвращает train, test, полный датасет и веса
  Если в датасете только уникальные пары user-item, то все веса = 1
  """

  train_with_duplicates, test_data = recs_split_random(data, test_size)
  all_data = data
  
  # Если создаем матрицу для implicit feedback (0, 1)
  if weight_col == 'items':
    # Отбираем столбцы с данными о пользователях и объектов рекомендации
    # Добавляем данные c частотой каждой пары user-item в поле 'items'
    train_data = (train_with_duplicates.groupby([user_col, item_col])
                  .size()
                  .to_frame('items')
                  .reset_index())
    
    all_data = (data.groupby([user_col, item_col])
                  .size()
                  .to_frame('items')
                  .reset_index()) 
    
  # Удаляем дубликаты в тестовых данных
  if delete_test_duplicates == True:    
      test_data = (test_data.groupby([user_col, item_col])
                    .size()
                    .to_frame('items')
                    .reset_index())  

  # Нормализуем данные в поле weight_col
  # train_weight = np.log2(train_data[weight_col] + 1)  
  # all_weight = np.log2(all_data[weight_col] + 1)
  # Не используем нормализацию
  train_weight = train_data[weight_col]
  all_weight = all_data[weight_col]

  # Отдельные датасеты для пар user-item 
  train_data = train_data[[user_col, item_col]]
  test_data = test_data[[user_col, item_col]]
  all_data = all_data[[user_col, item_col]]

  # Cписки уникальных пользователей для train и test
  train_users = np.sort(train_data[user_col].unique())
  test_users = np.sort(test_data[user_col].unique())

  # Списки уникальных items для train и test
  train_items= np.sort(train_data[item_col].unique())
  test_items= np.sort(test_data[item_col].unique())

  # Новые users и items, незнакомые для модели
  # они не попадут в обучение
  cold_start_items = set(test_items) - set(train_items)
  cold_start_users = set(test_users) - set(train_users)

  if print_report:
    print("=> Количество пользователей")
    print(f"Общий датасет: {len(unique_transactions[user_col].unique())}")
    print(f"Обучающая выборка: {len(train_users)}")
    print(f"Валидационная выборка: {len(test_users)}")
    print()    
    print("=> Проблема cold start для модели")
    print("Неизвестных пользователей:", len(cold_start_users))
    print("Неизвестных объектов:", len(cold_start_items))
  
  # Набор датасетов для обучения рекомендательных моделей
  recs_datasets = {
      'train_user_item': train_data,
      'test_user_item': test_data,
      'all_user_item': all_data,
      'train_weight': train_weight,
      # 'test_weight': test_weight,
      'all_weight': all_weight,

      # обучающая выборка с дубликатами
      'train_all': train_with_duplicates
  }
  return recs_datasets

def get_user_item_matrix(user_items_df, user_col, item_col, 
                         user_item_weight_list):

  """Генерирует СSR-матрицу user-items в формате индексов для обучения моделей
  
  Возвращает:
  -------
  csr-матрица user-items с индексами
  словари для обратного перевода индексов user и item в исходные названия
  """
  
  # Cловари для маппинга индексов ID -> читатель/книга
  index_to_user = pd.Series(np.sort(np.unique(user_items_df[user_col])))
  index_to_item = pd.Series(np.sort(np.unique(user_items_df[item_col])))

  # Словари для обратного маппинга
  user_to_index = pd.Series(data=index_to_user.index, 
                            index=index_to_user.values)
  item_to_index = pd.Series(data=index_to_item.index, 
                            index=index_to_item.values)

  # Добавляем индексы в обучающий датасет
  user_items_df_indexed = user_items_df.copy()
  user_items_df_indexed['user_id'] = user_items_df[user_col].map(user_to_index)
  user_items_df_indexed['item_id'] = user_items_df[item_col].map(item_to_index)

  # Данные для перевода датасета в CSR матрицу
  data = user_item_weight_list
  rows = user_items_df_indexed['user_id']
  cols = user_items_df_indexed['item_id']

  # CSR матрица user(строки)-books(столбцы)
  user_items_csr = csr_matrix((data, (rows, cols)), 
                              shape=(len(user_to_index), len(item_to_index)))

  return user_items_csr, index_to_user, index_to_item


########################################################################
# Получение рекомендаций с помощью моделей implicit
########################################################################

def get_implicit_recommendations(trained_model_imp, user_items_matrix, 
                                 index_to_user_map, index_to_item_map, 
                                 top_n=20, filter_liked_items=True):  
  """
  Генерирует top_n рекомендаций с обученной моделью библиотеки implicit

  Возвращает:
  ----------
  Датафрейм, где каждая строка - список top_n рекомендаций для пользователя
  индексы - названия users, 
  данные - названия items из словарей index_to_user_map, index_to_item_map
  """

  # Получаем список рекомендаций - матрица с индексами книг и читателей
  recs_matrix = trained_model_imp.recommend_all(
      user_items=user_items_matrix, 
      N=top_n, 
      filter_already_liked_items=filter_liked_items
  )

  # Создаем датафрейм
  # Переводим индексы в исходные названия user и item
  recs_df = (
      pd.DataFrame(recs_matrix, index=index_to_user_map)
      .apply(lambda c: c.map(index_to_item_map))
  )

  return recs_df


def get_avg_recs_metrics(recs_matrix, valid_user_items_df,
                               user_col, item_col, 
                               print_report=True, round_n=4):
  """
  Считает и выводит усредненные метрики для матрицы рекомендаций

  Параметры:
  ---------
  recs_matrix - матрица предсказаний, строка = список рекомендаций для пользователя
  valid_user_items_df - валидационный датасет 
  """
  # Создаем словарь из валидационного датасета
  # ключ = user_name, значения = item_name
  user_items_dict = (
      valid_user_items_df
      .groupby(user_col)[item_col]
      .apply(set).to_dict()
  )
  
  # Ищем пересечение множества пользователей, на которых была обучена модель
  # с пользователями в валидационном датасете
  train_users = recs_matrix.index.to_list()
  valid_users = valid_user_items_df[user_col].unique()
  users_to_compare = set(train_users) & set(valid_users)

  # Считаем усредненные метрики по всем пользователям
  # hit-rate, precision, recall и f1-score
  hitrate = round(
      np.mean([int(len(set(recs_matrix.loc[u]) & user_items_dict[u]) > 0) for u in users_to_compare]), round_n)

  precision = round(
      np.mean([
      len(set(recs_matrix.loc[u]) & user_items_dict[u]) / 
      len(recs_matrix.loc[u]) for u in users_to_compare]), 5)

  recall = round(
      np.mean([
      len(set(recs_matrix.loc[u]) & user_items_dict[u]) / 
      len(user_items_dict[u]) for u in users_to_compare]), 5)

  f1_score = round(
      2 * (precision * recall) / (precision + recall), 5)


  if print_report:
    print(f"Hit-Rate_avg: {hitrate}")
    print(f"Recall_avg: {precision}")
    print(f"Precision_avg: {recall}")
    print(f"F1-score_avg: {f1_score}")
    
  return [hitrate, precision, recall, f1_score]

#### Общий вывод

1. Мы подготовили более производительное решение для ALS 

  - Качество модели осталось без изменений

  - Подобрали новый набор метрик, который лучше отражает качество предсказаний для персональных рекомендаций
---
2. Протестировали новый подход:
  - Научились эффективно использовать дубликаты в данных, если использовать нормализацию


Теперь мы готовы испытать решение RankFM


### (2.2) RankFM - Factorization Machines

RankFM - это одна из библиотек, в которой реализованы методы Factorization Machines.

- альтернативы - библиотеки Implicit, LightFM 

Мы испытаем технику LTR, есть две loss-функции:
 - bpr (рекомендуют для implicit feedback, не зависит от весов)
 - warp

Посмотрим, какие результаты получим без добавления дополнительных признаков:)

Разработчики утверждают, что даже такой базовый вариант должен дать более высокий результат, чем ALS.

#### Подготовка данных

Все те же шаги, которые мы уже выполняли для метода ALS, только теперь воспользуемся готовыми функциями.

Главные отличия:

- для обучения не потребуются csr-матрицы
- нет необходимости для дополнительной индексации книг и читателей


In [ ]:
# Словарь с предобработанными датасетам для обучения
recs_dict = split_and_prepare_recs_data(unique_transactions, 
                                        'chb', 'sys_numb', 0.2)

=> Количество пользователей
Общий датасет: 16753
Обучающая выборка: 16668
Валидационная выборка: 12088

=> Проблема cold start для модели
Неизвестных пользователей: 85
Неизвестных объектов: 34727


#### Обучение модели и результаты


In [ ]:
model = RankFM(factors=200, 
               loss='warp', max_samples=3,  learning_rate=0.4)

model.fit(recs_dict['train_user_item'],          
          epochs=115,
          verbose=True)


training epoch: 0
log likelihood: -142373.84375

training epoch: 1
log likelihood: -75539.2109375

training epoch: 2
log likelihood: -36011.44140625

training epoch: 3
log likelihood: -25563.3203125

training epoch: 4
log likelihood: -21487.5

training epoch: 5
log likelihood: -19394.759765625

training epoch: 6
log likelihood: -18075.16015625

training epoch: 7
log likelihood: -17202.25

training epoch: 8
log likelihood: -16556.91015625

training epoch: 9
log likelihood: -16070.9404296875

training epoch: 10
log likelihood: -15681.2001953125

training epoch: 11
log likelihood: -15360.7802734375

training epoch: 12
log likelihood: -15100.259765625

training epoch: 13
log likelihood: -14874.7001953125

training epoch: 14
log likelihood: -14661.23046875

training epoch: 15
log likelihood: -14493.1103515625

training epoch: 16
log likelihood: -14328.759765625

training epoch: 17
log likelihood: -14198.23046875

training epoch: 18
log likelihood: -14074.849609375

training epoch: 19
log l

In [ ]:
# Сохраняем обученную модель

with open('rankfm_model_books', 'wb') as fle:
    pickle.dump(model, fle, protocol=pickle.HIGHEST_PROTOCOL)


**Вывод 1**

1. С каждой эпохой модель сходится все лучше, но очень небольшими шагами.

  - можно будет поработать над тюнингом параметров

2. Скорость обучения значительно более низкая, чем с ALS.

Стоила ли игра свеч, получим ли более высокие результаты??

In [ ]:
# Генерируем рекомендации
all_users_predictions = model.recommend(
    recs_dict['train_user_item'].chb.unique(), 
    n_items=20, filter_previous=True, cold_start='drop')

- Генерация рекомендаций заняла 14 минут!

In [ ]:
all_users_predictions.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
100000641403,RSL01004191071,RSL01007657281,RSL01004462230,RSL01003783075,RSL01007528691,RSL01004185702,RSL01007975629,RSL01004190969,RSL01003441482,RSL02000000555,RSL01004221786,RSL01003639696,RSL01004186500,RSL01008477654,RSL01003589346,RSL01002393437,RSL01004196026,RSL01007657422,RSL01003601634,RSL01004185091
100000644359,RSL01009839571,RSL01010705965,RSL01010247222,RSL07000458969,RSL01010278914,RSL01010247947,RSL01004937178,RSL01010559974,RSL01002304107,RSL01002884697,RSL01010582576,RSL01010553020,RSL01004230805,RSL01003172361,RSL01007901236,RSL01005411637,RSL07000489922,RSL01004096658,RSL01008299587,RSL01010723682
100000665127,RSL01008069959,RSL01010257833,RSL01001751573,RSL01010256248,RSL01003301462,RSL01003388020,RSL01004914143,RSL01005472644,RSL01002864722,RSL01009819710,RSL01010274592,RSL01010559359,RSL01006281612,RSL01003141778,RSL01005376787,RSL01003423392,RSL01003705868,RSL01003189375,RSL01004500694,RSL01003449085


- Получили рекомендации в том же формате, что и с методом recommend_all в Implicit


In [ ]:
# Переводим матрицу рекомендаций в датафрейм с парами user-book
recs_pairs = matrix_recs_to_pairs(all_users_predictions, 'chb', 'sys_numb') 

# Сохраняем результаты в файл
recs_pairs.to_csv("rankfm_bpr_books_recs.csv", index=False, sep=';')

- Проверим качество модели:)

In [ ]:
# Выводим усредненные показатели модели
get_avg_recs_metrics(all_users_predictions, 
                     recs_dict['test_user_item'],
                     'chb', 'sys_numb')

Hit-Rate_avg: 0.0247
Recall_avg: 0.00145
Precision_avg: 0.01062
F1-score_avg: 0.00255


[0.0247, 0.00145, 0.01062, 0.00255]

**Вывод**

1. Метрика ниже, а на генерацию рекомендаций у нас ушло 14 минут вместо 1 минуты)

2. Есть идея протестировать тот же алгоритм, но в реализации implicit:
  - здесь у нас уже протестированные подходы
  - единственное отличие, в implicit нет возможности показывать итерации обучения с помощью `verbose`

### (2.3) BPR - Реализация Implicit

#### Подготовка данных





In [ ]:
# Словарь с предобработанными датасетам для обучения
recs_dict = split_and_prepare_recs_data(unique_transactions, 
                                        'chb', 'sys_numb', 0.2)

=> Количество пользователей
Общий датасет: 16753
Обучающая выборка: 16661
Валидационная выборка: 12026

=> Проблема cold start для модели
Неизвестных пользователей: 92
Неизвестных объектов: 34695


#### Обучение модели

In [ ]:
# Формируем матрицы и словари с маппингом
user_books_csr, users_dict, books_dict= get_user_item_matrix(
    recs_dict['train_user_item'], 
    'chb', 'sys_numb', 
    recs_dict['train_weight'])                                                       

print("Размер sparse-матрицы user-books:", user_books_csr.shape)

Размер sparse-матрицы user-books: (16661, 159971)


In [ ]:
# Обучаем модель
model_bpr_imp = BayesianPersonalizedRanking(factors=200, learning_rate=0.3,  
                                            iterations=500, regularization=0.01)
model_bpr_imp.fit(user_books_csr, show_progress=True)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
# Получаем список top-20 рекомендаций для каждого читателя
recs_bpr_imp = get_implicit_recommendations(model_bpr_imp, user_books_csr,
                                            users_dict, books_dict)                                 
print("Предсказания модели ALS:")
recs_bpr_imp.head(3)

Предсказания модели ALS:


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
100000641403,RSL01001062381,RSL01004002279,RSL01004105688,RSL01004637327,RSL01004998543,RSL07000371522,RSL01002457362,RSL01003557141,RSL01003018189,RSL01003049752,RSL01005165940,RSL01008827912,RSL07000374511,RSL01008239763,RSL07000443962,RSL01000704642,RSL01004367600,RSL01009016247,RSL01004964344,RSL01003545222
100000644359,RSL01010765332,RSL01010247947,RSL01000693701,RSL01004096658,RSL01004937178,RSL01007901236,RSL01006589989,RSL01010256257,RSL01010252917,RSL01009839571,RSL01003842392,RSL07000458969,RSL01010617333,RSL01000069520,RSL01002768786,RSL07000494917,RSL01010559974,RSL01010723682,RSL07000489922,RSL01009477974
100000665127,RSL01001427316,RSL01003414117,RSL01004572877,RSL01003382900,RSL01009393690,RSL01003792464,RSL01009648400,RSL01001736815,RSL01008690841,RSL01010427794,RSL01008132136,RSL01007489421,RSL01009390694,RSL01008114610,RSL01008139459,RSL01006700657,RSL01001728161,RSL01003047920,RSL01010814838,RSL01002157583


#### Оценка результатов

In [ ]:
# Выводим усредненные показатели модели
get_avg_recs_metrics(recs_bpr_imp, recs_dict['test_user_item'],
                           'chb', 'sys_numb')

Hit-Rate_avg: 0.0218
Recall_avg: 0.0012
Precision_avg: 0.00704
F1-score_avg: 0.00205


[0.0218, 0.0012, 0.00704, 0.00205]

In [ ]:
# Переводим матрицу рекомендаций в датафрейм с парами user-book
solution = matrix_recs_to_pairs(recs_bpr_imp, 'chb', 'sys_numb') 

# Сохраняем предсказания
solution.to_csv("predictions_bpr_imp.csv", index=False, sep=';')

### (2.4) Общий вывод

1. Мы получили альтернативное решение для Colloborative Filtering:

  - Метод BPR обучается значительно быстрее, чем алгоритм ALS

  - Генерация рекомендаций очень медленно реализована в RankFM и LightFM, поэтому мы снова вернулись к библиотеке Implicit (26 минут vs 2 минуты)

2. Важно учитывать, что BPR не использует веса (это модель ранжирования):
  - параметр alpha для тюнинга использовать не можем
  
3. Также мы оптимизировали и подготовили новый набор функций для работы с рекомендательной системой. 

  - Работает быстро 

  - Можно использовать для любых моделей Implicit

  - Будет полезным для развития проекта и новых экспериментов)

(!) Есть идея отфильтровать аутлайеров - оставить в пуле только часто читаемые книги и активных пользователей